## BAŞVURU PUANI TAHMİNİ

İş Problemi: Başvuru puanını tahmin ederek başvuru puanının ne olacağını belirlemek.   
Hedef: Özelliklere dayalı olarak puan tahmin etmek için regresyon modelleri geliştirmek.


In [94]:
pip install pandas scikit-learn matplotlib seaborn


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.1 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [155]:
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.svm import SVR

print("Kurulumlar başarılı!")

Kurulumlar başarılı!


VERİ ÖN İŞLEME

In [96]:
# Excel dosyasını okuma
file_name = 'BASVURU.xlsx'
# DataFrame'i daha şık görüntüleme
dff = pd.read_excel(file_name, engine='openpyxl') 
# Tablo görüntüleme
dff.head()  


,KISI_ID,BASVURU_ID,YIL,DÖNEM,BASVURU_TIPI,UYRUK,CINSIYET,PUAN,BASARI_DURUMU,KESIN_KAYIT_DURUMU
0,NaN,9,2012,Güz,Uluslararası Öğrenci,İRAN,Kadın,15.0,DEGERLENDIRILMEDI,KESIN KAYIT YAPMADI
1,NaN,12,2012,Güz,Uluslararası Öğrenci,TÜRKMENİSTAN,Kadın,5.0,DEGERLENDIRILMEDI,KESIN KAYIT YAPMADI
2,NaN,16,2012,Güz,Uluslararası Öğrenci,AZERBAYCAN,Erkek,1430.0,DEGERLENDIRILMEDI,KESIN KAYIT YAPMADI
3,NaN,20,2012,Güz,Uluslararası Öğrenci,SURİYE,Erkek,220.0,DEGERLENDIRILMEDI,KESIN KAYIT YAPMADI
4,NaN,26,2012,Güz,Uluslararası Öğrenci,TÜRKMENİSTAN,Kadın,76.0,DEGERLENDIRILMEDI,KESIN KAYIT YAPMADI


In [97]:
# Veri setinin boyutlarını öğrenme
print(dff.shape)

(17548, 10)


In [98]:
# Sütun adlarını ve veri tiplerini görüntüleme
print(dff.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17548 entries, 0 to 17547
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   KISI_ID             1026 non-null   float64
 1   BASVURU_ID          17548 non-null  int64  
 2   YIL                 17548 non-null  int64  
 3   DÖNEM               17548 non-null  object 
 4   BASVURU_TIPI        17548 non-null  object 
 5   UYRUK               17548 non-null  object 
 6   CINSIYET            17548 non-null  object 
 7   PUAN                17548 non-null  float64
 8   BASARI_DURUMU       17548 non-null  object 
 9   KESIN_KAYIT_DURUMU  17548 non-null  object 
dtypes: float64(2), int64(2), object(6)
memory usage: 1.3+ MB
None


In [99]:
# Veri setinin ilk istatistiksel özetini alma
print(dff.describe())


            KISI_ID    BASVURU_ID           YIL          PUAN
count  1.026000e+03  17548.000000  17548.000000  17548.000000
mean   8.479358e+05   6126.239913   2013.821860    114.259668
std    3.680203e+05   3208.415247      0.635975    259.577873
min    3.300000e+01      5.000000   2010.000000      0.000000
25%    1.007124e+06   3857.000000   2014.000000     45.500000
50%    1.007330e+06   6019.500000   2014.000000     67.165000
75%    1.007775e+06   8918.250000   2014.000000     81.250000
max    1.009997e+06  12326.000000   2015.000000   7168.000000


In [100]:
# Hatalı sonuç alamamak adına 'KISI_ID', 'BASVURU_ID' ve 'YIL' sütunlarını kategorik veriye çevirme ve görüntüleme
dff['KISI_ID'] = dff['KISI_ID'].astype('category')
dff['BASVURU_ID'] = dff['BASVURU_ID'].astype('category')
dff['YIL'] = dff['YIL'].astype('category')
print(dff.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17548 entries, 0 to 17547
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype   
---  ------              --------------  -----   
 0   KISI_ID             1026 non-null   category
 1   BASVURU_ID          17548 non-null  category
 2   YIL                 17548 non-null  category
 3   DÖNEM               17548 non-null  object  
 4   BASVURU_TIPI        17548 non-null  object  
 5   UYRUK               17548 non-null  object  
 6   CINSIYET            17548 non-null  object  
 7   PUAN                17548 non-null  float64 
 8   BASARI_DURUMU       17548 non-null  object  
 9   KESIN_KAYIT_DURUMU  17548 non-null  object  
dtypes: category(3), float64(1), object(6)
memory usage: 1.4+ MB
None


In [101]:
# Veri setinin istatistiksel özetini alma
print(dff.describe())

               PUAN
count  17548.000000
mean     114.259668
std      259.577873
min        0.000000
25%       45.500000
50%       67.165000
75%       81.250000
max     7168.000000


In [102]:
# Eksik verileri kontrol etme
print(dff.isnull().sum())

KISI_ID               16522
BASVURU_ID                0
YIL                       0
DÖNEM                     0
BASVURU_TIPI              0
UYRUK                     0
CINSIYET                  0
PUAN                      0
BASARI_DURUMU             0
KESIN_KAYIT_DURUMU        0
dtype: int64


In [103]:
# Eksik verileri belirli sütunlar için kontrol etme
for col in dff.columns:
    missing_count = dff[col].isnull().sum()
    if missing_count > 0:
        print(f"{col} sütununda {missing_count} eksik veri var.")


KISI_ID sütununda 16522 eksik veri var.


In [104]:
# KISI_ID sütununu atma
dff = dff.drop(columns=['KISI_ID'])

# İlk birkaç satırı görüntüleme ve sütunun atıldığını doğrulama
print(dff.head())


  BASVURU_ID   YIL DÖNEM          BASVURU_TIPI         UYRUK CINSIYET    PUAN  \
0          9  2012   Güz  Uluslararası Öğrenci          İRAN    Kadın    15.0   
1         12  2012   Güz  Uluslararası Öğrenci  TÜRKMENİSTAN    Kadın     5.0   
2         16  2012   Güz  Uluslararası Öğrenci    AZERBAYCAN    Erkek  1430.0   
3         20  2012   Güz  Uluslararası Öğrenci        SURİYE    Erkek   220.0   
4         26  2012   Güz  Uluslararası Öğrenci  TÜRKMENİSTAN    Kadın    76.0   

       BASARI_DURUMU   KESIN_KAYIT_DURUMU  
0  DEGERLENDIRILMEDI  KESIN KAYIT YAPMADI  
1  DEGERLENDIRILMEDI  KESIN KAYIT YAPMADI  
2  DEGERLENDIRILMEDI  KESIN KAYIT YAPMADI  
3  DEGERLENDIRILMEDI  KESIN KAYIT YAPMADI  
4  DEGERLENDIRILMEDI  KESIN KAYIT YAPMADI  


In [105]:
# Güncellenmiş veri setinin kaydedilmesi ve görüntülenmesi
dff.to_csv('cleaned_data_without_id.csv', index=False)
dff.head()

,BASVURU_ID,YIL,DÖNEM,BASVURU_TIPI,UYRUK,CINSIYET,PUAN,BASARI_DURUMU,KESIN_KAYIT_DURUMU
0,9,2012,Güz,Uluslararası Öğrenci,İRAN,Kadın,15.0,DEGERLENDIRILMEDI,KESIN KAYIT YAPMADI
1,12,2012,Güz,Uluslararası Öğrenci,TÜRKMENİSTAN,Kadın,5.0,DEGERLENDIRILMEDI,KESIN KAYIT YAPMADI
2,16,2012,Güz,Uluslararası Öğrenci,AZERBAYCAN,Erkek,1430.0,DEGERLENDIRILMEDI,KESIN KAYIT YAPMADI
3,20,2012,Güz,Uluslararası Öğrenci,SURİYE,Erkek,220.0,DEGERLENDIRILMEDI,KESIN KAYIT YAPMADI
4,26,2012,Güz,Uluslararası Öğrenci,TÜRKMENİSTAN,Kadın,76.0,DEGERLENDIRILMEDI,KESIN KAYIT YAPMADI


In [106]:
# Sütun adlarını görüntüleme
print("Orijinal Sütunlar:", dff.columns)

Orijinal Sütunlar: Index(['BASVURU_ID', 'YIL', 'DÖNEM', 'BASVURU_TIPI', 'UYRUK', 'CINSIYET',
       'PUAN', 'BASARI_DURUMU', 'KESIN_KAYIT_DURUMU'],
      dtype='object')


In [107]:
# Veri setindeki kategorik ve sayısal değişkenleri ayırma
categorical_cols = dff.select_dtypes(include=['object', 'category']).columns
numerical_cols = dff.select_dtypes(include=['number']).columns

print(f"Kategorik Sütunlar: {categorical_cols}")
print(f"Sayısal Sütunlar: {numerical_cols}")

Kategorik Sütunlar: Index(['BASVURU_ID', 'YIL', 'DÖNEM', 'BASVURU_TIPI', 'UYRUK', 'CINSIYET',
       'BASARI_DURUMU', 'KESIN_KAYIT_DURUMU'],
      dtype='object')
Sayısal Sütunlar: Index(['PUAN'], dtype='object')


In [108]:
# Hata kontrolü için kategorik veri içeren sütunların benzersiz değerlerinin görüntülenmesi
unique_values_donem = dff['DÖNEM'].unique()
print("Benzersiz Dönem Değerleri:", unique_values_donem)


unique_values_tip = dff['BASVURU_TIPI'].unique()
print("Benzersiz Başvuru Tipi Değerleri:", unique_values_tip)

unique_values_uyruk = dff['UYRUK'].unique()
print("Benzersiz Uyruk Değerleri:", unique_values_uyruk)

unique_values_cinsiyet = dff['CINSIYET'].unique()
print("Benzersiz Cinsiyet Değerleri:", unique_values_cinsiyet)

unique_values_basari = dff['BASARI_DURUMU'].unique()
print("Benzersiz Başarı Durumu Değerleri:", unique_values_basari)

unique_values_kayit = dff['KESIN_KAYIT_DURUMU'].unique()
print("Benzersiz Kesin Kayıt Durumu Değerleri:", unique_values_kayit)


Benzersiz Dönem Değerleri: ['Güz' 'Bahar']
Benzersiz Başvuru Tipi Değerleri: ['Uluslararası Öğrenci' 'Enstitu' 'Lisans Yatay Geçiş']
Benzersiz Uyruk Değerleri: ['İRAN' 'TÜRKMENİSTAN' 'AZERBAYCAN' 'SURİYE' 'FİLİSTİN' 'YUNANİSTAN'
 'İNGİLTERE / BİRLEŞİK KRALLIK' 'ALMANYA' 'UKRAYNA' 'TACİKİSTAN'
 'ARNAVUTLUK' 'ÖZBEKİSTAN' 'TÜRKİYE' 'MORİTANYA' 'A.B.D.'
 'KUZEY MAKEDONYA' 'KUZEY KIBRIS TÜRK CUMHURİYETİ' 'IRAK' 'KIRGIZİSTAN'
 'AFGANİSTAN' 'AVUSTURYA' 'KENYA' 'KOSOVA' 'MISIR' 'BULGARİSTAN'
 'TANZANYA' 'GÜRCİSTAN' 'SIRBİSTAN' 'FRANSA' 'ÇİN' 'GÜNEY AFRİKA'
 'PAKİSTAN' 'RUSYA' 'ÜRDÜN' 'ROMANYA' 'HOLLANDA' 'SOMALİ' 'ENDONEZYA'
 'SUDAN' 'DANİMARKA' 'GİNE BİSSAU' 'FAS' 'İSRAİL' 'UGANDA'
 'SUUDİ ARABİSTAN' 'ÇAD' 'YEMEN' 'NİJERYA' 'SENEGAL' 'KAMERUN' 'LİBYA'
 'İTALYA' 'TUNUS' 'MALİ' 'GİNE' 'KAZAKİSTAN' 'İSVİÇRE' 'PERU' 'MOLDOVA'
 'GÜNEY KIBRIS RUM KESİMİ' 'BOSNA HERSEK' 'KANADA' 'ETİOPYA' 'KARADAĞ'
 'AMERİKAN SAMOA' 'MOĞOLİSTAN' 'AVUSTRALYA' 'BELÇİKA' 'GANA' 'İSVEÇ'
 'FİLİPİNLER' 'PAPUA YENİ GİNE' '

In [109]:
# Cinsiyet sütununu yakından inceleme
value_counts_cinsiyet = dff['CINSIYET'].value_counts()
print("Cinsiyet Değerlerinin Sayımları:")
print(value_counts_cinsiyet)

Cinsiyet Değerlerinin Sayımları:
CINSIYET
Erkek         10755
Kadın          6769
Bilinmeyen       24
Name: count, dtype: int64


In [110]:
# Bilinmeyen cinsiyet değerlerine sahip satırları filtreleme
df = dff[dff['CINSIYET'] != 'Bilinmeyen']

# Filtrelenmiş veri setini kontrol etme
print(df['CINSIYET'].value_counts())


CINSIYET
Erkek    10755
Kadın     6769
Name: count, dtype: int64


In [113]:
# Güncellenmiş veri setini kaydetme ve görüntüleme
df.to_csv('corrected_data.csv', index=False)
df

,BASVURU_ID,YIL,DÖNEM,BASVURU_TIPI,UYRUK,CINSIYET,PUAN,BASARI_DURUMU,KESIN_KAYIT_DURUMU
0,9,2012,Güz,Uluslararası Öğrenci,İRAN,Kadın,15.00,DEGERLENDIRILMEDI,KESIN KAYIT YAPMADI
1,12,2012,Güz,Uluslararası Öğrenci,TÜRKMENİSTAN,Kadın,5.00,DEGERLENDIRILMEDI,KESIN KAYIT YAPMADI
2,16,2012,Güz,Uluslararası Öğrenci,AZERBAYCAN,Erkek,1430.00,DEGERLENDIRILMEDI,KESIN KAYIT YAPMADI
3,20,2012,Güz,Uluslararası Öğrenci,SURİYE,Erkek,220.00,DEGERLENDIRILMEDI,KESIN KAYIT YAPMADI
4,26,2012,Güz,Uluslararası Öğrenci,TÜRKMENİSTAN,Kadın,76.00,DEGERLENDIRILMEDI,KESIN KAYIT YAPMADI
...,...,...,...,...,...,...,...,...,...
17543,12204,2015,Güz,Enstitu,TÜRKİYE,Kadın,75.44,Kazandı,KESIN KAYIT YAPTI
17544,12187,2015,Güz,Enstitu,TÜRKİYE,Kadın,76.50,Kazandı,KESIN KAYIT YAPTI
17545,12231,2015,Güz,Enstitu,TÜRKİYE,Kadın,76.90,Kazandı,KESIN KAYIT YAPTI
17546,12104,2015,Güz,Enstitu,TÜRKİYE,Erkek,77.77,Kazandı,KESIN KAYIT YAPTI


In [125]:
# Veri setinin boyutlarını öğrenme
print(df.shape)

(17524, 9)


In [126]:
# Özellikler ve hedef değişkeni ayırma
X = df.drop(columns=['PUAN'])
y = df['PUAN']

In [127]:
# Kategorik ve sayısal veri ön işleme adımları
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(), categorical_cols)
    ],
    remainder='passthrough'  # Kategorikleri kategorik olarak diğerlerini olduğu gibi geçirme
)

In [128]:
# Ön işleme işlemlerini veri üzerinde uygulama
X_preprocessed = preprocessor.fit_transform(X)

MODEL SEÇİMİ VE EĞİTİM

In [133]:
# Eğitim ve test setlerine ayırma
X_train, X_test, y_train, y_test = train_test_split(X_preprocessed, y, test_size=0.2, random_state=42)

In [134]:
# Model listesini oluşturma
models = {
    'Linear Regression': LinearRegression(),
    'Ridge Regression': Ridge(),
    'Lasso Regression': Lasso(),
    'Random Forest Regressor': RandomForestRegressor(),
    'Gradient Boosting Regressor': GradientBoostingRegressor(),
    'SVR': SVR()
}

In [135]:
# Modelleri eğitim ve değerlendirme
results = {}
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    results[name] = mse

In [136]:
# Sonuçları yazdırma
for name, mse in results.items():
    print(f"{name} MSE: {mse}")

Linear Regression MSE: 102350.28535826648
Ridge Regression MSE: 74316.55089038973
Lasso Regression MSE: 63793.36976206791
Random Forest Regressor MSE: 90746.73659855081
Gradient Boosting Regressor MSE: 63855.997605760684
SVR MSE: 76967.26945256816


En düşük MSE değerlerine sahip olan Lasso ve Gradient Boosting için Hiperparametre Tunning ve Model İyileştirme

In [139]:
# Lasso için Hiperparametre Ayarları
# Hiperparametre ızgarası
param_grid_lasso = {
    'alpha': [0.001, 0.01, 0.1, 1, 10, 100]
}

In [140]:
# Grid Search oluşturma
grid_search_lasso = GridSearchCV(Lasso(), param_grid=param_grid_lasso, cv=5, scoring='neg_mean_squared_error')
grid_search_lasso.fit(X_train, y_train)

c:\Users\elift\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:658: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 278735.6941482723, tolerance: 76472.24716437994
  model = cd_fast.sparse_enet_coordinate_descent(
c:\Users\elift\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:658: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2842570.8586623967, tolerance: 76651.52270953935
  model = cd_fast.sparse_enet_coordinate_descent(


GridSearchCV(cv=5, estimator=Lasso(),
             param_grid={'alpha': [0.001, 0.01, 0.1, 1, 10, 100]},
             scoring='neg_mean_squared_error')

In [141]:
# En iyi hiperparametreler ve model
best_params_lasso = grid_search_lasso.best_params_
best_lasso_model = grid_search_lasso.best_estimator_

print(f"En İyi Lasso Hiperparametreler: {best_params_lasso}")

En İyi Lasso Hiperparametreler: {'alpha': 0.1}


In [142]:
# En iyi model ile tahmin yapma
y_pred_lasso = best_lasso_model.predict(X_test)

In [143]:
# Gradient Boosting için Hiperparametre Ayarı
# Hiperparametre ızgarası
param_grid_gbr = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7]
}

In [144]:
# Grid Search oluşturma
grid_search_gbr = GridSearchCV(GradientBoostingRegressor(), param_grid=param_grid_gbr, cv=5, scoring='neg_mean_squared_error')
grid_search_gbr.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=GradientBoostingRegressor(),
             param_grid={'learning_rate': [0.01, 0.1, 0.2],
                         'max_depth': [3, 5, 7],
                         'n_estimators': [50, 100, 200]},
             scoring='neg_mean_squared_error')

In [145]:
# En iyi hiperparametreler ve model
best_params_gbr = grid_search_gbr.best_params_
best_gbr_model = grid_search_gbr.best_estimator_

print(f"En İyi Gradient Boosting Hiperparametreler: {best_params_gbr}")

En İyi Gradient Boosting Hiperparametreler: {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 50}


In [146]:
# En iyi model ile tahmin yapma
y_pred_gbr = best_gbr_model.predict(X_test)

In [147]:
# Sonuçların Karşılaştırılması

# En iyi Lasso modelinin MSE'si
mse_lasso = mean_squared_error(y_test, y_pred_lasso)
print(f"En İyi Lasso Modeli MSE: {mse_lasso}")

# En iyi Gradient Boosting modelinin MSE'si
mse_gbr = mean_squared_error(y_test, y_pred_gbr)
print(f"En İyi Gradient Boosting Modeli MSE: {mse_gbr}")

En İyi Lasso Modeli MSE: 59498.36709500305
En İyi Gradient Boosting Modeli MSE: 63319.8518615505


LASSO REGRESSION

In [152]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 17524 entries, 0 to 17547
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype   
---  ------              --------------  -----   
 0   BASVURU_ID          17524 non-null  category
 1   YIL                 17524 non-null  category
 2   DÖNEM               17524 non-null  object  
 3   BASVURU_TIPI        17524 non-null  object  
 4   UYRUK               17524 non-null  object  
 5   CINSIYET            17524 non-null  object  
 6   PUAN                17524 non-null  float64 
 7   BASARI_DURUMU       17524 non-null  object  
 8   KESIN_KAYIT_DURUMU  17524 non-null  object  
dtypes: category(2), float64(1), object(6)
memory usage: 1.4+ MB


In [154]:
# Kategorik veri türlerinin sayısal veriye dönüştürülmesi (Kategorik verilerin encodingi)
# Kategorik sütunları seç
categorical_features = ['BASVURU_ID', 'YIL', 'DÖNEM', 'BASVURU_TIPI', 'UYRUK', 'CINSIYET', 'BASARI_DURUMU', 'KESIN_KAYIT_DURUMU']

# One-Hot Encoding uygula
encoder = OneHotEncoder(drop='first', sparse_output=False)  # drop='first' ilk kategoriyi kaldırarak dummy variable trap'ı önler, sparse_output=True, varsayılan olarak sparse matrix döndürür
encoded_features = encoder.fit_transform(dff[categorical_features])

# DataFrame'e dönüştür
encoded_df = pd.DataFrame(encoded_features, columns=encoder.get_feature_names_out(categorical_features))

# Orijinal ve encode edilmiş verileri birleştir
df_encoded = pd.concat([dff.drop(columns=categorical_features), encoded_df], axis=1)


In [156]:
# Özellik ölçekleme

# Sayısal sütunları seç
numerical_features = ['PUAN']

# Özellikleri ölçekle
scaler = StandardScaler()
scaled_features = scaler.fit_transform(df_encoded[numerical_features])

# DataFrame'e dönüştür
scaled_df = pd.DataFrame(scaled_features, columns=numerical_features)

# Orijinal ve ölçeklenmiş verileri birleştir
df_final = pd.concat([df_encoded.drop(columns=numerical_features), scaled_df], axis=1)


In [157]:
df

,BASVURU_ID,YIL,DÖNEM,BASVURU_TIPI,UYRUK,CINSIYET,PUAN,BASARI_DURUMU,KESIN_KAYIT_DURUMU
0,9,2012,Güz,Uluslararası Öğrenci,İRAN,Kadın,15.00,DEGERLENDIRILMEDI,KESIN KAYIT YAPMADI
1,12,2012,Güz,Uluslararası Öğrenci,TÜRKMENİSTAN,Kadın,5.00,DEGERLENDIRILMEDI,KESIN KAYIT YAPMADI
2,16,2012,Güz,Uluslararası Öğrenci,AZERBAYCAN,Erkek,1430.00,DEGERLENDIRILMEDI,KESIN KAYIT YAPMADI
3,20,2012,Güz,Uluslararası Öğrenci,SURİYE,Erkek,220.00,DEGERLENDIRILMEDI,KESIN KAYIT YAPMADI
4,26,2012,Güz,Uluslararası Öğrenci,TÜRKMENİSTAN,Kadın,76.00,DEGERLENDIRILMEDI,KESIN KAYIT YAPMADI
...,...,...,...,...,...,...,...,...,...
17543,12204,2015,Güz,Enstitu,TÜRKİYE,Kadın,75.44,Kazandı,KESIN KAYIT YAPTI
17544,12187,2015,Güz,Enstitu,TÜRKİYE,Kadın,76.50,Kazandı,KESIN KAYIT YAPTI
17545,12231,2015,Güz,Enstitu,TÜRKİYE,Kadın,76.90,Kazandı,KESIN KAYIT YAPTI
17546,12104,2015,Güz,Enstitu,TÜRKİYE,Erkek,77.77,Kazandı,KESIN KAYIT YAPTI
